In [1]:
import time,os,requests,io
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from scipy import optimize


PFMs = {"SBI":"001", "HDFC":"008", "ICICI":"007"}
tiers = {"E1":1, "E2":4}
offsets = {"SBI":2, "HDFC":0, "ICICI":0}

mypfm = "SBI"
mytier = "E1"

### Read Input & add Nifty Data

In [2]:
# Read the Transactions File
tdf = pd.read_csv("npsTransactions.csv")
def negUnits(x):
    try:
        x = float(x)
    except:
        if(x[0] == "("):
            x = -float(x[1:-1])
        else:
            print("error")
    return x
tdf['Amt'] = tdf['Amt'].apply(negUnits)

tdf['Date'] = pd.to_datetime(tdf['Date'])
tdf = tdf.sort_values(by='Date',ascending=True, ignore_index=True)
tdf['Date'] = tdf['Date'].dt.strftime('%Y-%m-%d')

In [3]:
def getNifty(date):
	url="https://archives.nseindia.com/content/indices/ind_close_all_%s.csv"%date
	s=requests.get(url).content
	indicesDf=pd.read_csv(io.StringIO(s.decode('utf-8')))
	indicesDf = indicesDf.loc[indicesDf['Index Name'] == 'Nifty 50']
	niftyClose = indicesDf.at[0,'Closing Index Value']
	return float(niftyClose/100)

df = tdf
for i in range(len(df['Date'])):
    inv = float(df.at[i,'Amt'])
    date = datetime.strptime(df.at[i,'Date'], "%Y-%m-%d")
    try:
        if(str(df.at[i,"Nifty NAV"]) == "nan"): 
            print("Nifty",date)
            df.at[i,"Nifty NAV"] = getNifty(date.strftime("%d%m%Y"))
    except:
        date = date - timedelta(days=1)
        print("Nifty-1",date)
        df.at[i,"Nifty NAV"] = getNifty(date.strftime("%d%m%Y"))

df = pd.read_csv("npsTransactions.csv")
df["Nifty NAV"] = tdf["Nifty NAV"]
#df.to_csv("npsTransactions.csv",index=False)

Nifty-1 2022-04-08 00:00:00
Nifty 2022-04-20 00:00:00
Nifty 2022-06-13 00:00:00
Nifty 2022-07-09 00:00:00
Nifty-1 2022-07-08 00:00:00
Nifty 2022-07-18 00:00:00
Nifty 2022-08-19 00:00:00
Nifty 2022-09-01 00:00:00
Nifty 2023-03-13 00:00:00
Nifty 2023-04-06 00:00:00
Nifty 2023-04-11 00:00:00
Nifty 2023-05-08 00:00:00
Nifty 2023-06-12 00:00:00


### Compute

In [4]:
tdate = datetime.today()
fdate = tdate - timedelta(days = 7)

dfNAV = pd.DataFrame()
for pfm in PFMs.keys():
    for tier in tiers.keys():
        o = requests.get(f"https://www.npstrust.org.in/nav-graphs-details?lnavdata=PFM{PFMs[pfm]}&yearval={fdate.strftime('%Y-%m-%d')}@{tdate.strftime('%Y-%m-%d')}&subcat=SM{PFMs[pfm]}00{offsets[pfm]+tiers[tier]}")

        date = []
        nav = []
        for d in o.text[2:-2].split("],["):
            d = d.split(",")
            date.append(datetime.fromtimestamp(int(d[0][:-3])))
            nav.append(d[1])

        dfNAV["Date"] = pd.to_datetime(date).strftime("%Y-%m-%d")
        dfNAV[f"{pfm}_{tier}"] = nav

In [5]:
# Initialization and functions
def pfmNAV(date,pfm,tier="E1"):
    return float(dfNAV.loc[(dfNAV["Date"]) == "%s"%date][f"{pfm}_{tier}"]) 

# Functions
def xnpv(rate,cashflows):
    chron_order = sorted(cashflows, key = lambda x: x[0])
    t0 = chron_order[0][0]
    return sum([float(cf)/(1+rate)**((t-t0).days/365.0) for (t,cf) in chron_order])

def xirr(cashflows,currDate,current,guess=0.1):
	for i in range(len(cashflows)):
		cashflows[i][0] = datetime.strptime(cashflows[i][0], "%Y-%m-%d") 
	cashflows.append((currDate,-current))
	return optimize.newton(lambda r: xnpv(r,cashflows),guess)

In [6]:
df = tdf.copy()
df["NPS Units"] = df["Amt"]/df["NAV"]
df["Nifty Units"] = df["Amt"]/df["Nifty NAV"]

In [7]:
# Compute Present Value
df['NPS Units'] = df['NPS Units'].apply(lambda x: round(x, 4))
df['Nifty Units'] = df['Nifty Units'].apply(lambda x: round(x, 4))

invested = df['Amt'].sum()
hdfcUnits = df['NPS Units'].sum()
niftyUnits = df['Nifty Units'].sum()

dfNAV["Date"] = pd.to_datetime(dfNAV["Date"])
tdate = list(dfNAV["Date"])[-1]
nifNAV = getNifty(tdate.strftime("%d%m%Y"))

nav = pfmNAV(tdate,pfm=mypfm,tier=mytier)
hdfcCurr = hdfcUnits*float(nav)
niftyCurr = niftyUnits*nifNAV

xirrHDFC = 100*xirr(((df[["Date", "Amt"]]).values.tolist()),tdate,hdfcCurr)
xirrNIFTY = 100*xirr(((df[["Date", "Amt"]]).values.tolist()),tdate,niftyCurr)

index = len(df['Date'])
df.at[index,"Date"] = tdate.strftime("%d-%b-%Y")
df.at[index,"Amt"] = invested
df.at[index,"NAV"] = nav
df.at[index,"Nifty NAV"] = nifNAV
df.at[index,"NPS Units"] = round(hdfcCurr,2)
df.at[index,"Nifty Units"] = round(niftyCurr,2)
df.at[index+1,"NPS Units"] = str(round(xirrHDFC,2))+"%"
df.at[index+1,"Nifty Units"] = str(round(xirrNIFTY,2))+"%"

In [8]:
# Save to file
df.to_csv("NPS_XIRR.csv",index=False)